In [1]:

import pickle 
##Load data
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('/Users/abhisheksinghal/Documents/garima/MLLearning/ubi/supp.csv')

df.head()

,row_id,time_id,investment_id,f_0,f_1,f_2,f_3,f_4,f_5,f_6,...,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299,target
0,0_2470,0,2470,0.857476,0.014525,0.788850,0.723694,0.110336,0.872259,0.622063,...,0.207291,0.672806,0.543164,0.140206,0.874127,0.608734,0.002161,0.777650,0.777969,0.482648
1,1_3175,1,3175,0.136743,0.365311,0.480053,0.486144,0.947280,0.828768,0.467746,...,0.719423,0.019378,0.253559,0.959608,0.044995,0.075371,0.052130,0.799216,0.392113,0.870882
2,2_1427,2,1427,0.303859,0.905065,0.653210,0.803277,0.324553,0.209632,0.801480,...,0.239102,0.654874,0.877775,0.882025,0.389150,0.995435,0.220038,0.463028,0.548430,0.135198
3,3_2842,3,2842,0.310167,0.184374,0.250553,0.408470,0.625977,0.783919,0.461546,...,0.131791,0.905971,0.647910,0.164506,0.953576,0.928850,0.617431,0.966846,0.199578,0.765026
4,4_1183,4,1183,0.751231,0.512202,0.216569,0.874850,0.786549,0.428286,0.004994,...,0.057409,0.355222,0.897738,0.475867,0.918221,0.638254,0.713265,0.827862,0.682437,0.553994


In [2]:
y = df.target
x = df
x = x.drop(['target', 'row_id'], axis=1)
print(x.shape)
print(y.shape)

(942961, 302)
(942961,)


In [5]:
filename = "/Users/abhisheksinghal/Documents/garima/MLLearning/ubi/mar_model.sav"

zsh:1: command not found: pip


In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F

In [9]:
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

tabnet_params = dict(n_d=24, n_a=24, n_steps=1, gamma=1.3,
                     lambda_sparse=0, optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     mask_type='entmax',
                     scheduler_params=dict(mode="min",
                                           patience=5,
                                           min_lr=1e-5,
                                           factor=0.9,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=10,
                     )
regressor = TabNetRegressor(**tabnet_params)

# load the model from disk
regressor = pickle.load(open(filename, 'rb'))


Device used : cpu


/var/folders/6l/f406sbm938jcwm7939s8dl900000gn/T/ipykernel_48445/919412889.py:19: DeprecationWarning: Please use `csc_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csc` namespace is deprecated.
  regressor = pickle.load(open(filename, 'rb'))


In [ ]:
tscv = TimeSeriesSplit(n_splits = 20)
rmse = []
CV_score_array    =[]
predictions_array =[]

for train_index, test_index in tscv.split(x):
    X_train, X_valid = x.iloc[train_index], x.iloc[test_index]
    y_train, y_valid = y[train_index], y[test_index]

    X_train      = X_train.to_numpy()
    y_train = y_train.to_numpy().reshape(-1, 1)
    
    X_valid      = X_valid.to_numpy()
    y_valid = y_valid.to_numpy().reshape(-1, 1)
    
    regressor.fit(X_train=X_train, y_train=y_train,
              eval_set=[(X_valid, y_valid)],
              patience=300, max_epochs=100,
              eval_metric=['rmse'])

    predictions = regressor.predict(X_valid)
    pickle.dump(regressor, open(filename, 'wb'))
    CV_score_array.append(regressor.best_cost)
    print("The CV score is %.5f" % np.mean(CV_score_array,axis=0) )
    predictions_array.append(np.expm1(predictions))
    
    

Stop training because you reached max_epochs = 100 with best_epoch = 30 and best_val_0_rmse = 0.28884
Best weights from best epoch are automatically used!
The CV score is 0.28884
Stop training because you reached max_epochs = 100 with best_epoch = 98 and best_val_0_rmse = 0.2886
Best weights from best epoch are automatically used!
The CV score is 0.28872
Stop training because you reached max_epochs = 100 with best_epoch = 44 and best_val_0_rmse = 0.2891
Best weights from best epoch are automatically used!
The CV score is 0.28885
Stop training because you reached max_epochs = 100 with best_epoch = 72 and best_val_0_rmse = 0.28896
Best weights from best epoch are automatically used!
The CV score is 0.28888
Stop training because you reached max_epochs = 100 with best_epoch = 24 and best_val_0_rmse = 0.28919
Best weights from best epoch are automatically used!
The CV score is 0.28894
Stop training because you reached max_epochs = 100 with best_epoch = 30 and best_val_0_rmse = 0.28898
Best 

In [ ]:
import pickle
filename = 'mar_model.sav'
pickle.dump(regressor, open(filename, 'wb'))


In [ ]:
# load the model from disk
regressor = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)